<a href="https://colab.research.google.com/github/daniyal9538/CID/blob/master/count_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code iterates through a range of files, counts up data points in each file and outputs it into 2 csv files. The code downloads a file from storage bucket, iterates through it and counts it up, and then deletes that file.
Once all the files are counted, the 2 csv files are downloaded to the local pc


In [0]:
import numpy as np 
import csv

In [0]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'test-demo-222423'
bucket_name = 'cid_training_data'
# Create the service client.
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

from apiclient.http import MediaIoBaseDownload
count = [0]*9

In [0]:
for i in range(1, 6):
  tempCount = [0]*9
  try: 
    download_name = "training_data-{}.npy".format(i)
    path_name = "balanced_data/balanced_data-{}.npy".format(i)

    with open(download_name, 'wb') as f:
      # Download the file from a given Google Cloud Storage bucket.
      request = gcs_service.objects().get_media(bucket=bucket_name,
                                                object=path_name)
      media = MediaIoBaseDownload(f, request)

      done = False
      while not done:
        # _ is a placeholder for a progress object that we ignore.
        # (Our file is small, so we skip reporting progress.)
        _, done = media.next_chunk()        

    print('Download complete ', i)
    
    try:
      data = np.load(download_name)
      for x in data:
        t = np.argmax(x[1])
        count[t]+=1
        tempCount[t]+=1
      
      
      print (tempCount)
      sum =0
      for d in tempCount:
          sum+=d
      
      print(count)
      s=[100*j/sum for j in tempCount]
      print (s)
      with open('dataD.csv', 'a') as fd:
          writer = csv.writer(fd)
          writer.writerow(s)
      with open('dataC.csv', 'a') as fd:
          writer = csv.writer(fd)
          writer.writerow(tempCount)
          
          
    except Exception as ex:
        print("Could not count data {}".format(i), ex)
    print('Count complete', i)
    
    !rm -rf $download_name
    
  except Exception as e:
    !rm -rf $download_name
    print("Could not download download file {}".format(i), e)
    

Download complete  1
[1111, 1111, 1111, 1111, 1111, 1111, 1046, 992, 1111]
[1111, 1111, 1111, 1111, 1111, 1111, 1046, 992, 1111]
[11.319409067753439, 11.319409067753439, 11.319409067753439, 11.319409067753439, 11.319409067753439, 11.319409067753439, 10.657157412124299, 10.10697911360163, 11.319409067753439]
Count complete 1
Download complete  2
[1111, 1111, 1111, 1111, 1111, 1111, 1046, 992, 1111]
[2222, 2222, 2222, 2222, 2222, 2222, 2092, 1984, 2222]
[11.319409067753439, 11.319409067753439, 11.319409067753439, 11.319409067753439, 11.319409067753439, 11.319409067753439, 10.657157412124299, 10.10697911360163, 11.319409067753439]
Count complete 2
Download complete  3
[1111, 1111, 1111, 1111, 1111, 1111, 1046, 992, 1111]
[3333, 3333, 3333, 3333, 3333, 3333, 3138, 2976, 3333]
[11.319409067753439, 11.319409067753439, 11.319409067753439, 11.319409067753439, 11.319409067753439, 11.319409067753439, 10.657157412124299, 10.10697911360163, 11.319409067753439]
Count complete 3
Download complete  4

In [0]:
from google.colab import files
#files.download('dataD.csv')
files.download('dataC.csv')